In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix

In [2]:
# loading the directories 
training_dir = '../input/cabbagetypescnn/cabbage'

In [3]:
# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 5


In [4]:
# importing the libraries
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16
#from keras.preprocessing import image

IMAGE_SIZE = [150, 150]  # we will keep the image size as (64,64). You can increase the size for better results. 

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
#x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



2022-07-19 08:33:35.485938: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 08:33:35.591784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 08:33:35.592681: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 08:33:35.594501: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 0s 0us/step


In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=IMAGE_SIZE,
    batch_size=16,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    training_dir, # same directory as training data
    target_size=IMAGE_SIZE,
    batch_size=16,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 80 images belonging to 5 classes.
Found 20 images belonging to 5 classes.


In [7]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 16,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 16,
    epochs = 8)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-07-19 08:33:40.081519: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/8


2022-07-19 08:33:41.939781: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


5/5 [==============================] - 10s 491ms/step - loss: 1.7434 - accuracy: 0.2250 - val_loss: 1.3500 - val_accuracy: 0.3125
Epoch 2/8
5/5 [==============================] - 2s 362ms/step - loss: 1.1505 - accuracy: 0.5000 - val_loss: 1.1312 - val_accuracy: 0.6250
Epoch 3/8
5/5 [==============================] - 2s 313ms/step - loss: 0.7841 - accuracy: 0.7000 - val_loss: 0.7600 - val_accuracy: 0.6875
Epoch 4/8
5/5 [==============================] - 2s 346ms/step - loss: 0.4926 - accuracy: 0.9000 - val_loss: 0.6602 - val_accuracy: 0.8125
Epoch 5/8
5/5 [==============================] - 2s 468ms/step - loss: 0.3338 - accuracy: 0.9500 - val_loss: 0.6549 - val_accuracy: 0.6875
Epoch 6/8
5/5 [==============================] - 2s 346ms/step - loss: 0.2611 - accuracy: 0.9625 - val_loss: 0.5168 - val_accuracy: 0.8125
Epoch 7/8
5/5 [==============================] - 2s 343ms/step - loss: 0.2445 - accuracy: 0.9500 - val_loss: 0.4055 - val_accuracy: 0.9375
Epoch 8/8
5/5 [=====================

In [8]:
# The labels are stored in class_indices in dictionary form. 
# checking the labels
train_generator.class_indices

{'bokchoy': 0, 'green': 1, 'napa': 2, 'red': 3, 'savoy': 4}

In [9]:
#training_images = 82
#validation_images = 20

#history = model.fit_generator(train_generator,
  #                 steps_per_epoch = 82,  # this should be equal to total number of images in training set. But to speed up the execution, I am only using 10000 images. Change this for better results. 
    #               epochs = 8,  # change this for better results
   #                validation_data = validation_generator)
      #             validation_steps = 20)  # this should be equal to total number of images in validation set.

In [10]:
print ('Training Accuracy = ' + str(history.history['accuracy']))
print ('Validation Accuracy = ' + str(history.history['val_accuracy']))

Training Accuracy = [0.22499999403953552, 0.5, 0.699999988079071, 0.8999999761581421, 0.949999988079071, 0.9624999761581421, 0.949999988079071, 0.9750000238418579]
Validation Accuracy = [0.3125, 0.625, 0.6875, 0.8125, 0.6875, 0.8125, 0.9375, 0.9375]
